<a href="https://www.kaggle.com/code/towlei/titanic-nn-with-knn-imputations?scriptVersionId=157831797" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

In [3]:
train_data=pd.read_csv("/kaggle/input/titanic/train.csv")
test_data=pd.read_csv("/kaggle/input/titanic/test.csv")

In [4]:
train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [5]:
nan_count = train_data.isna().sum()
print(nan_count)

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64


In [6]:
train_data.Embarked.unique()
train_data['Embarked_s']=train_data['Embarked'].replace('Q',0).replace('C',0).replace('S',1)
train_data['Embarked_c']=train_data['Embarked'].replace('S',0).replace('Q',0).replace('C',1)
train_data['Embarked_q']=train_data['Embarked'].replace('C',0).replace('S',0).replace('Q',1)

In [7]:
test_data.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [8]:
test_data.Embarked.unique()
test_data['Embarked_s']=test_data['Embarked'].replace('Q',0).replace('C',0).replace('S',1)
test_data['Embarked_c']=test_data['Embarked'].replace('S',0).replace('Q',0).replace('C',1)
test_data['Embarked_q']=test_data['Embarked'].replace('C',0).replace('S',0).replace('Q',1)

In [9]:
train_data['Sex'].replace( 'female', 0, inplace=True )
train_data['Sex'].replace( 'male', 1, inplace=True )
train_data= train_data.drop(columns=['Embarked'])
train_data

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked_s,Embarked_c,Embarked_q
0,1,0,3,"Braund, Mr. Owen Harris",1,22.0,1,0,A/5 21171,7.2500,NaN,1.0,0.0,0.0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,38.0,1,0,PC 17599,71.2833,C85,0.0,1.0,0.0
2,3,1,3,"Heikkinen, Miss. Laina",0,26.0,0,0,STON/O2. 3101282,7.9250,NaN,1.0,0.0,0.0
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,35.0,1,0,113803,53.1000,C123,1.0,0.0,0.0
4,5,0,3,"Allen, Mr. William Henry",1,35.0,0,0,373450,8.0500,NaN,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",1,27.0,0,0,211536,13.0000,NaN,1.0,0.0,0.0
887,888,1,1,"Graham, Miss. Margaret Edith",0,19.0,0,0,112053,30.0000,B42,1.0,0.0,0.0
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",0,NaN,1,2,W./C. 6607,23.4500,NaN,1.0,0.0,0.0
889,890,1,1,"Behr, Mr. Karl Howell",1,26.0,0,0,111369,30.0000,C148,0.0,1.0,0.0


In [10]:
test_data['Sex'].replace( 'female', 0, inplace=True )
test_data['Sex'].replace( 'male', 1, inplace=True )
test_data= test_data.drop(columns=['Embarked'])
test_data

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked_s,Embarked_c,Embarked_q
0,892,3,"Kelly, Mr. James",1,34.5,0,0,330911,7.8292,NaN,0,0,1
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",0,47.0,1,0,363272,7.0000,NaN,1,0,0
2,894,2,"Myles, Mr. Thomas Francis",1,62.0,0,0,240276,9.6875,NaN,0,0,1
3,895,3,"Wirz, Mr. Albert",1,27.0,0,0,315154,8.6625,NaN,1,0,0
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",0,22.0,1,1,3101298,12.2875,NaN,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,"Spector, Mr. Woolf",1,NaN,0,0,A.5. 3236,8.0500,NaN,1,0,0
414,1306,1,"Oliva y Ocana, Dona. Fermina",0,39.0,0,0,PC 17758,108.9000,C105,0,1,0
415,1307,3,"Saether, Mr. Simon Sivertsen",1,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,1,0,0
416,1308,3,"Ware, Mr. Frederick",1,NaN,0,0,359309,8.0500,NaN,1,0,0


In [11]:
train_knn = train_data.copy(deep=True)
test_knn = test_data.copy(deep=True)

In [12]:
from sklearn.impute import KNNImputer
train_knn = train_data.copy(deep=True)

knn_imputer = KNNImputer(n_neighbors=2, weights="uniform")
train_knn['Age'] = knn_imputer.fit_transform(train_knn[['Age']])

In [13]:
from sklearn.impute import KNNImputer
test_knn = test_data.copy(deep=True)

knn_imputer = KNNImputer(n_neighbors=2, weights="uniform")
test_knn['Age'] = knn_imputer.fit_transform(test_knn[['Age']])

In [14]:
test_knn['Age'].isnull().sum()

0

In [15]:
train_knn['Age'].isnull().sum()

0

In [16]:
train_knn.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked_s       2
Embarked_c       2
Embarked_q       2
dtype: int64

In [17]:
train_knn=train_knn.dropna(subset=['Embarked_s','Embarked_c','Embarked_q'])

In [18]:
test_knn['Fare'].fillna(0, inplace=True)
test_knn.isnull().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          327
Embarked_s       0
Embarked_c       0
Embarked_q       0
dtype: int64

In [19]:
train_knn

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked_s,Embarked_c,Embarked_q
0,1,0,3,"Braund, Mr. Owen Harris",1,22.000000,1,0,A/5 21171,7.2500,NaN,1.0,0.0,0.0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,38.000000,1,0,PC 17599,71.2833,C85,0.0,1.0,0.0
2,3,1,3,"Heikkinen, Miss. Laina",0,26.000000,0,0,STON/O2. 3101282,7.9250,NaN,1.0,0.0,0.0
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,35.000000,1,0,113803,53.1000,C123,1.0,0.0,0.0
4,5,0,3,"Allen, Mr. William Henry",1,35.000000,0,0,373450,8.0500,NaN,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",1,27.000000,0,0,211536,13.0000,NaN,1.0,0.0,0.0
887,888,1,1,"Graham, Miss. Margaret Edith",0,19.000000,0,0,112053,30.0000,B42,1.0,0.0,0.0
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",0,29.699118,1,2,W./C. 6607,23.4500,NaN,1.0,0.0,0.0
889,890,1,1,"Behr, Mr. Karl Howell",1,26.000000,0,0,111369,30.0000,C148,0.0,1.0,0.0


In [20]:
test_knn

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked_s,Embarked_c,Embarked_q
0,892,3,"Kelly, Mr. James",1,34.50000,0,0,330911,7.8292,NaN,0,0,1
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",0,47.00000,1,0,363272,7.0000,NaN,1,0,0
2,894,2,"Myles, Mr. Thomas Francis",1,62.00000,0,0,240276,9.6875,NaN,0,0,1
3,895,3,"Wirz, Mr. Albert",1,27.00000,0,0,315154,8.6625,NaN,1,0,0
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",0,22.00000,1,1,3101298,12.2875,NaN,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,"Spector, Mr. Woolf",1,30.27259,0,0,A.5. 3236,8.0500,NaN,1,0,0
414,1306,1,"Oliva y Ocana, Dona. Fermina",0,39.00000,0,0,PC 17758,108.9000,C105,0,1,0
415,1307,3,"Saether, Mr. Simon Sivertsen",1,38.50000,0,0,SOTON/O.Q. 3101262,7.2500,NaN,1,0,0
416,1308,3,"Ware, Mr. Frederick",1,30.27259,0,0,359309,8.0500,NaN,1,0,0


In [21]:
y = train_knn['Survived']

features = ['Pclass', 'Age', 'Fare','Sex', 'Embarked_c', 'Embarked_q', 'Embarked_s']
X = pd.get_dummies(train_knn[features])
X_test = pd.get_dummies(test_knn[features])

print(X_test)

     Pclass       Age      Fare  Sex  Embarked_c  Embarked_q  Embarked_s
0         3  34.50000    7.8292    1           0           1           0
1         3  47.00000    7.0000    0           0           0           1
2         2  62.00000    9.6875    1           0           1           0
3         3  27.00000    8.6625    1           0           0           1
4         3  22.00000   12.2875    0           0           0           1
..      ...       ...       ...  ...         ...         ...         ...
413       3  30.27259    8.0500    1           0           0           1
414       1  39.00000  108.9000    0           1           0           0
415       3  38.50000    7.2500    1           0           0           1
416       3  30.27259    8.0500    1           0           0           1
417       3  30.27259   22.3583    1           1           0           0

[418 rows x 7 columns]


In [22]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X)
X_test = scaler.transform(X_test)

In [23]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(256, activation='relu', input_dim=X_train.shape[1]),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [24]:
history = model.fit(X_train, y, epochs=200, batch_size=32, validation_split=0.2)

Epoch 1/200
23/23 [==============================] - 2s 14ms/step - loss: 0.6270 - accuracy: 0.6723 - val_loss: 0.5014 - val_accuracy: 0.8258
Epoch 2/200
23/23 [==============================] - 0s 4ms/step - loss: 0.5031 - accuracy: 0.7792 - val_loss: 0.4107 - val_accuracy: 0.7921
Epoch 3/200
23/23 [==============================] - 0s 4ms/step - loss: 0.4980 - accuracy: 0.7609 - val_loss: 0.3998 - val_accuracy: 0.8315
Epoch 4/200
23/23 [==============================] - 0s 5ms/step - loss: 0.4650 - accuracy: 0.7961 - val_loss: 0.4006 - val_accuracy: 0.8202
Epoch 5/200
23/23 [==============================] - 0s 4ms/step - loss: 0.4658 - accuracy: 0.8017 - val_loss: 0.3995 - val_accuracy: 0.8371
Epoch 6/200
23/23 [==============================] - 0s 4ms/step - loss: 0.4684 - accuracy: 0.8087 - val_loss: 0.3963 - val_accuracy: 0.8258
Epoch 7/200
23/23 [==============================] - 0s 4ms/step - loss: 0.4512 - accuracy: 0.8101 - val_loss: 0.3946 - val_accuracy: 0.8539
Epoch 8/200


In [25]:
predictions = model.predict(X_test)
predictions = np.round(predictions).astype(int).reshape(-1)

submission = pd.DataFrame({
    'PassengerId': test_data['PassengerId'],
    'Survived': predictions
})

# Save the submission file
submission.to_csv('titanic_submission.csv', index=False)

14/14 [==============================] - 0s 2ms/step


In [26]:
sub=pd.read_csv("./titanic_submission.csv")
sub

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0
